In [1]:
import h_transport_materials as htm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from analysis_functions import parse_ufloat
from uncertainties import unumpy as unp

/opt/anaconda3/envs/festim2-env/lib/python3.13/site-packages/pybtex/plugin/__init__.py:26: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# material_str = "316l_steel"
material_str = "steel"

# filter only 316l_steel and H
diffusivities = htm.diffusivities.filter(material=material_str).filter(isotope="h")
permeabilities = htm.permeabilities.filter(material=material_str).filter(isotope="h")

In [3]:
# --- Edit this path ---
csv_path = f"results/figs/{material_str}_diffusivities.csv"

# --- Read and clean data ---
df = pd.read_csv(csv_path)

# Apply parser to relevant columns
for col in ["Temperature (K)", "Diffusivity", "Permeability"]:
    df[col] = df[col].apply(parse_ufloat)

# Drop rows with missing data
df = df.dropna(subset=["Temperature (K)", "Diffusivity", "Permeability"])

# Sort by the nominal value of Temperature
df = df.sort_values(by="Temperature (K)", key=lambda s: [v.n if hasattr(v, "n") else v for v in s])

# Extract numeric arrays or ufloats
T = df["Temperature (K)"].to_numpy()
D = df["Diffusivity"].to_numpy()
Phi = df["Permeability"].to_numpy()
P_up = df["Upstream Pressure (Torr)"].to_numpy()

# get nominal and std arrays for plotting
T_nom = [t.n if hasattr(t, "n") else t for t in T]
D_nom = [d.n if hasattr(d, "n") else d for d in D]
Phi_nom = [p.n if hasattr(p, "n") else p for p in Phi]
D_std = [d.s if hasattr(d, "s") else 0 for d in D]
Phi_std = [p.s if hasattr(p, "s") else 0 for p in Phi]


[540.88162909 553.47422537 554.8271342 ]


In [5]:
T = 550
dT = 20
low = T - dT
high = T + dT


T_nom = np.array(T_nom)
P_up = np.array(P_up)
Phi_nom = np.array(Phi_nom)

valid_T_nom = (T_nom >= low) & (T_nom <= high)
idx = np.where(valid_T_nom)[0]

print(T_nom[idx])

plt.figure()
plt.plot(P_up[idx], Phi_nom[idx], 'o')
plt.ylabel("Permeability")
plt.xlabel("Upstream Pressure (Torr)")
plt.show()
